<a href="https://colab.research.google.com/github/lychee1223/Swallow/blob/main/Swallow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title ライブラリの準備

!pip install sentencepiece accelerate protobuf bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [2]:
# @title モデルの準備

# モデルの選択
model_name = "tokyotech-llm/Swallow-13b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']

# モデルのダウンロード
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = "auto",
    device_map = "auto",
    trust_remote_code = True,
    low_cpu_mem_usage = True,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [3]:
# @title プロンプトのテンプレートを定義

PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input = None):
    if input:
        return PROMPT_DICT["prompt_input"].format(instruction = instruction, input = input)
    else:
        return PROMPT_DICT["prompt_no_input"].format(instruction = instruction)


In [4]:
# @title プロンプトを作成

instruction = "以下のトピックに関する詳細な情報を提供してください。" # @param {type:"string"}
input = "UnityとUnreal Engineそれぞれの特徴について" #@param {type:"string"}

prompt = create_prompt(instruction = instruction, input = input)


In [5]:
# @title 推論

temperature = 0.2 # @param {type:"slider", min:0.1, max:1, step:0.1}
top_p = 0.95 # @param {type:"slider", min:0.01, max:1, step:0.01}

with torch.no_grad():
    input_ids = tokenizer.encode(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    tokens = model.generate(
        input_ids.to(device=model.device),
        temperature = temperature,
        top_p = top_p,
        max_new_tokens=256,
        do_sample=True,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens = True)
print(output)


以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
UnityとUnreal Engineそれぞれの特徴について

### 応答:Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、ゲーム開発に特化したソフトウェアです。Unityは、
